In [ ]:
import glob
import numpy as np
import pickle
import torch
import importlib
import webdataset as wds
import random
import h5py
import matplotlib.pyplot as plt 
import wandb

# Configurations

In [ ]:
importlib.reload(plot)
plot.plot(results_folder="./train_logs/", 
     exp_name="pretrainNOV16_1700670984.431539*", 
                    num_runs_analyze=10000)

In [ ]:
importlib.reload(plot)
plot.plot_learning_curve(results_folder="./train_logs/", 
     exp_name="finetune_subj1_perf_vs_numsession_NOV24*", 
                    num_runs_analyze=10)

# Pretraining 

In [ ]:
importlib.reload(plot)
plot.plot_pretraining(results_folder="./train_logs/", 
     exp_name="pretrainNOV16_1700670984*", 
                    num_runs_analyze=100000)

# Wandb

In [ ]:
import wandb
import seaborn as sns
api = wandb.Api()
ENTITY = "anhhuyalex"
PROJECT = "eye_int"
# Filter runs by group name
runs = api.runs(f"{ENTITY}/{PROJECT}")#, filters={"group": "finetune_numsessions"})

# Download specific data from each run
num_sessions = []
test_bwd_pct_correct = []
test_fwd_pct_correct = []
for run in runs:
    if run.state == "finished" and run.config.get("num_sessions", None) is not None:
        num_sessions.append(   run.config["num_sessions"])
        test_bwd_pct_correct.append( run.summary["test/test_bwd_pct_correct"])
        test_fwd_pct_correct.append( run.summary["test/test_fwd_pct_correct"])
sns.set_theme(style="whitegrid")
ax = sns.lineplot( x=num_sessions, y=test_fwd_pct_correct, marker="o", label="test_fwd_pct_correct")
ax = sns.lineplot( x=num_sessions, y=test_bwd_pct_correct, marker="o", label="test_bwd_pct_correct")

ax.set_title("test_fwd_pct_correct")
ax.legend(loc="lower right")
ax.set_xlabel("num_sessions")

In [ ]:
import wandb
import seaborn as sns
api = wandb.Api()
ENTITY = "anhhuyalex"
PROJECT = "eye_int"
# Filter runs by group name
runs = api.runs(f"{ENTITY}/{PROJECT}", filters={"group": "finetune_numsamples"})

# get samples pool
filter_samples = "./cache/subj01_permuted_samples.txt"
with open(filter_samples) as f:
    samples_pool = f.read().splitlines()
 
# Download specific data from each run
num_samples = []
left_out = []
test_bwd_pct_correct = []
test_fwd_pct_correct = []
for run in runs:
    if run.state == "finished" and run.config.get("num_samples", None) == 6999:
        samples_list = run.config["samples_list"]
        left_out.append([x for x in samples_pool[:7000] if x not in samples_list])
        test_bwd_pct_correct.append( run.history().iloc[19]["test/test_bwd_pct_correct"])
        test_fwd_pct_correct.append( run.history().iloc[19]["test/test_fwd_pct_correct"])
        
sns.set_theme(style="whitegrid")
ax = sns.histplot(  x=test_fwd_pct_correct,   label="test_fwd_pct_correct")
ax = sns.histplot( x=test_bwd_pct_correct,  label="test_bwd_pct_correct")


ax.legend(loc="lower right")


In [13]:
import wandb
import seaborn as sns
api = wandb.Api()
ENTITY = "anhhuyalex"
PROJECT = "eye_int"
# Filter runs by group name
runs = api.runs(f"{ENTITY}/{PROJECT}", filters={"group": "check_variance_of_run"})

# get samples pool
filter_samples = "./cache/subj01_permuted_samples.txt"
with open(filter_samples) as f:
    samples_pool = f.read().splitlines()
 
# Download specific data from each run
num_samples = []
left_out = []
test_bwd_pct_correct = []
test_fwd_pct_correct = []
for run in runs:
    # run.delete()
#     if run.state == "finished" and run.config.get("num_samples", None) == 6999:
    samples_list = run.config["samples_list"]
    left_out.append([x for x in samples_pool[:7000] if x not in samples_list])
    print("left out", left_out[-1])
    try:
        test_bwd_pct_correct.append( run.history().iloc[19]["test/test_bwd_pct_correct"])
        test_fwd_pct_correct.append( run.history().iloc[19]["test/test_fwd_pct_correct"])
    except:
        continue
sns.set_theme(style="whitegrid")
ax = sns.histplot(  x=test_fwd_pct_correct,   label="test_fwd_pct_correct")
ax = sns.histplot( x=test_bwd_pct_correct,  label="test_bwd_pct_correct")


ax.legend(loc="lower right")


left out ['sample000003988']
left out ['sample000003988']
left out ['sample000003988']
left out ['sample000003988']
left out ['sample000003988']
left out ['sample000003988']
left out ['sample000003988']
left out ['sample000003988']
left out ['sample000003988']
left out ['sample000003988']
left out ['sample000003988']
left out ['sample000003988']
left out ['sample000003988']
left out ['sample000003988']
left out ['sample000003988']
left out ['sample000003988']
left out ['sample000003988']
left out ['sample000003988']
left out ['sample000003988']
left out ['sample000003988']


KeyboardInterrupt: 

In [ ]:
worst_performing_prunes_bwd = [left_out[i] for i in np.argsort(test_bwd_pct_correct)[:25]]
best_performing_prunes_bwd = [left_out[i] for i in np.argsort(test_bwd_pct_correct)[-25:]]
print(worst_performing_prunes_bwd)
print(best_performing_prunes_bwd)
print([i for i in worst_performing_prunes_bwd if i in best_performing_prunes_bwd])

worst_performing_prunes_fwd = [left_out[i] for i in np.argsort(test_fwd_pct_correct)[:25]]
best_performing_prunes_fwd = [left_out[i] for i in np.argsort(test_fwd_pct_correct)[-25:]]
print(worst_performing_prunes_fwd)
print(best_performing_prunes_fwd)
print([i for i in best_performing_prunes_fwd if i in worst_performing_prunes_fwd])

In [ ]:
data_path="/fsx/proj-fmri/shared/mindeyev2_dataset"
train_url = f"{data_path}/wds/subj01/train/" + "{0.." + f"39" + "}.tar"
print(f"subj01 with {train_url} train_url")
def my_split_by_node(urls): return urls
def filter_samples_func(sample):
    if sample["__key__"] not in worst_performing_prunes:
        return None
    return sample
train_data = wds.WebDataset(train_url,resampled=True,nodesplitter=my_split_by_node)\
                    .decode("torch") \
                    .rename(behav="behav.npy", past_behav="past_behav.npy", future_behav="future_behav.npy", olds_behav="olds_behav.npy")\
                    .to_tuple(*["__key__","behav", "past_behav", "future_behav", "olds_behav"])
train_dl = torch.utils.data.DataLoader(train_data, batch_size=1, shuffle=False, drop_last=True, pin_memory=True)
data_type = torch.float16
f = h5py.File(f'{data_path}/coco_images_224_float16.hdf5', 'r')
images = f['images'][:]
images = torch.Tensor(images).to("cpu").to(data_type)

In [ ]:
# BACKWARD PERFORMANCE IS WORST IF THESE IMAGES OMITTED
sns.set_style("white")
seen_images = set()
fig, ax = plt.subplots(5, 5, figsize=(15, 15))
worst_images = {}
for iter, (key, behav0, past_behav0, future_behav0, old_behav0) in enumerate(train_dl):  
    if key[0] in seen_images: continue
    try:
        rank = worst_performing_prunes_bwd.index(key)
    except:
        continue
    
    seen_images.add(key[0])
    worst_images[rank] = images[behav0[:,0,0].cpu().long()].float()
    if len(seen_images) == 25: break
for i in range(25):
    ax[i%5,i//5].imshow(worst_images[i][0].permute(1,2,0))
plt.show()

In [ ]:
# BACKWARD PERFORMANCE IS WORST IF THESE IMAGES OMITTED
sns.set_style("white")
seen_images = set()
fig, ax = plt.subplots(5, 5, figsize=(15, 15))
worst_images = {}
for iter, (key, behav0, past_behav0, future_behav0, old_behav0) in enumerate(train_dl):  
    if key[0] in seen_images: continue
    try:
        rank = worst_performing_prunes_fwd.index(key)
    except:
        continue
    
    seen_images.add(key[0])
    worst_images[rank] = images[behav0[:,0,0].cpu().long()].float()
    if len(seen_images) == 25: break
for i in range(25):
    ax[i%5,i//5].imshow(worst_images[i][0].permute(1,2,0))
plt.show()

In [ ]:
# BACKWARD PERFORMANCE IS WORST IF THESE IMAGES OMITTED
sns.set_style("white")
seen_images = set()
fig, ax = plt.subplots(5, 5, figsize=(15, 15))
best_images = {}
for iter, (key, behav0, past_behav0, future_behav0, old_behav0) in enumerate(train_dl):  
    if key[0] in seen_images: continue
    try:
        rank = best_performing_prunes_bwd.index(key)
    except:
        continue
    
    seen_images.add(key[0])
    best_images[rank] = images[behav0[:,0,0].cpu().long()].float()
    if len(seen_images) == 25: break
for i in range(25):
    ax[i%5,i//5].imshow(best_images[i][0].permute(1,2,0))
plt.show()

In [ ]:
# BACKWARD PERFORMANCE IS WORST IF THESE IMAGES OMITTED
sns.set_style("white")
seen_images = set()
fig, ax = plt.subplots(5, 5, figsize=(15, 15))
best_images = {}
for iter, (key, behav0, past_behav0, future_behav0, old_behav0) in enumerate(train_dl):  
    if key[0] in seen_images: continue
    try:
        rank = best_performing_prunes_fwd.index(key)
    except:
        continue
    
    seen_images.add(key[0])
    best_images[rank] = images[behav0[:,0,0].cpu().long()].float()
    if len(seen_images) == 25: break
for i in range(25):
    ax[i%5,i//5].imshow(best_images[i][0].permute(1,2,0))
plt.show()

In [ ]:
from functools import partial
from diffusers.models.vae import Decoder
class BrainNetwork(nn.Module):
    def __init__(self, out_dim=768, in_dim=15724, clip_size=768, h=4096, n_blocks=4, norm_type='ln', act_first=False, drop=.15, blurry_dim=16):
        super().__init__()
        self.blurry_dim = blurry_dim
        norm_func = partial(nn.BatchNorm1d, num_features=h) if norm_type == 'bn' else partial(nn.LayerNorm, normalized_shape=h)
        act_fn = partial(nn.ReLU, inplace=True) if norm_type == 'bn' else nn.GELU
        act_and_norm = (act_fn, norm_func) if act_first else (norm_func, act_fn)
        self.lin0 = nn.Linear(in_dim, h)
        self.mlp = nn.ModuleList([
            nn.Sequential(
                nn.Linear(h, h),
                *[item() for item in act_and_norm],
                nn.Dropout(drop)
            ) for _ in range(n_blocks)
        ])
        self.lin1 = nn.Linear(h, out_dim, bias=True)
        self.blin1 = nn.Linear(out_dim, blurry_dim, bias=True)
        self.n_blocks = n_blocks
        self.clip_size = clip_size
        self.clip_proj = nn.Sequential(
            nn.LayerNorm(clip_size),
            nn.GELU(),
            nn.Linear(clip_size, 2048),
            nn.LayerNorm(2048),
            nn.GELU(),
            nn.Linear(2048, 2048),
            nn.LayerNorm(2048),
            nn.GELU(),
            nn.Linear(2048, clip_size)
        )
        self.upsampler = Decoder(
                in_channels=64,
                out_channels=4,
                up_block_types=["UpDecoderBlock2D","UpDecoderBlock2D","UpDecoderBlock2D"],
                block_out_channels=[64, 128, 256],
                layers_per_block=1,
            )
        
    def forward(self, x):
        x = self.lin0(x)
        residual = x
        for res_block in range(self.n_blocks):
            x = self.mlp[res_block](x)
            x += residual
            residual = x
        x = x.reshape(len(x), -1)
        x = self.lin1(x)
        b = self.blin1(x)
        b = self.upsampler(b.reshape(len(b), -1, 7, 7))
        c = self.clip_proj(x.reshape(len(x), -1, self.clip_size))
        return c, b

model.backbone = BrainNetwork(h=2048, in_dim=hidden_dim, clip_size=clip_emb_dim, out_dim=clip_emb_dim*clip_seq_dim, blurry_dim=64*7*7) 
utils.count_params(model.backbone)
utils.count_params(model)

b = torch.randn((2,hidden_dim))
print(b.shape)
clip_, blur_ = model.backbone(b)
print(clip_.shape, blur_.shape)

In [ ]:
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
opt_grouped_parameters = [
    {'params': [p for n, p in model.ridge.named_parameters()], 'weight_decay': 1e-2},
    {'params': [p for n, p in model.backbone.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 1e-2},
    {'params': [p for n, p in model.backbone.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
]

optimizer = torch.optim.AdamW(opt_grouped_parameters, lr=max_lr, betas=(0.9, 0.95))

if lr_scheduler_type == 'linear':
    lr_scheduler = torch.optim.lr_scheduler.LinearLR(
        optimizer,
        total_iters=int(num_epochs*(num_train*num_devices//batch_size)),
        last_epoch=-1
    )
elif lr_scheduler_type == 'cycle':
    total_steps=int(num_epochs*(num_train*num_devices//batch_size))
    lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer, 
        max_lr=max_lr,
        total_steps=total_steps,
        final_div_factor=1000,
        last_epoch=-1, pct_start=2/num_epochs
    )
    
def save_ckpt(tag):    
    ckpt_path = outdir+f'/{tag}.pth'
    print(f'saving {ckpt_path}',flush=True)
    unwrapped_model = accelerator.unwrap_model(model)
    try:
        torch.save({
            'epoch': epoch,
            'model_state_dict': unwrapped_model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'lr_scheduler': lr_scheduler.state_dict(),
            'train_losses': losses,
            'test_losses': test_losses,
            'lrs': lrs,
            }, ckpt_path)
    except:
        print("Couldn't save... moving on to prevent crashing.")
    del unwrapped_model
        
print("\nDone with model preparations!")
utils.count_params(model)

# Active learning prune by 100

In [1]:
import webdataset as wds 
import random
import torch
def my_split_by_node(urls): return urls

voxels = {}
for si, s in enumerate([1]):
    train_url = f"/weka/proj-fmri/shared/mindeyev2_dataset/wds/subj0{s}/train/" + "{0.." + f"39" + "}.tar"
    print(f"subj01 training with {train_url} train_url")
    # Load hdf5 data for betas, but don't put everything into memory
    f = h5py.File(f'/weka/proj-fmri/shared/mindeyev2_dataset/betas_all_subj0{s}_fp32.hdf5', 'r')
    betas = f['betas'][:]
    del f
train_url = "/weka/proj-fmri/shared/mindeyev2_dataset/wds/subj01/new_train/{0..39}.tar"
train_data = wds.WebDataset(train_url,resampled=True,nodesplitter=my_split_by_node)\
                        .shuffle(750, initial=2500, rng=random.Random(1))\
                        .decode("torch")\
                       .rename(behav="behav.npy", past_behav="past_behav.npy", future_behav="future_behav.npy", olds_behav="olds_behav.npy")\
                       .to_tuple(*["__key__", "behav", "past_behav", "future_behav", "olds_behav"]) 
train_dl = torch.utils.data.DataLoader(train_data, batch_size=512, shuffle=False, drop_last=True, pin_memory=True) 
keys = set() 
# for iter, (key) in enumerate(train_dl):    
for iter, (key, behav0, past_behav0, future_behav0, old_behav0) in enumerate(train_dl): 
    
#     print (behav0[:,0,0].cpu().long())
#     print (past_behav0[:,0,0].cpu().long())
    [keys.add( i.item() ) for i in behav0[:,0,0].cpu().long()]
#     print ((keys))
    if iter % 10 == 1: print (len(keys))
    if iter > 1000: break

830
3942
5985
6976
7593
7937
8316
8518
8668
8866
8899
8931
8931
8931
8964
8964
8964
8964
8964
9000
9000


KeyboardInterrupt: 

In [6]:

import copy
import numpy as np
keys = np.load("./cache/training_set.npy")
np.random.shuffle(keys) 
print (len(keys), keys[:20])
num_keys_chosen = 100
for seed in range(len(keys)//num_keys_chosen):
    print ("len", len(keys[seed * num_keys_chosen: (seed+1) * num_keys_chosen]))
    np.save (f"./cache/keys_{num_keys_chosen}_from_0_seed_{seed}.npy", keys[seed * num_keys_chosen: (seed+1) * num_keys_chosen])
    

8000 [41540  3184 39692  7591 65417 45166 53785 63090 62372 58812 67111 25527
 34318 70615 45635 10631 51451 48697 61180 61875]
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100
len 100


In [2]:
import numpy as np
t = np.load("./cache/training_set.npy")
v = np.load("./cache/validation_set.npy")
assert not any(_t in set(v) for _t in t), "Found an element of t in v" 

t, v

(array([42121, 41062, 14842, ..., 33656, 15692, 36417]),
 array([50203, 30596, 72074, 49510, 67375,  1886, 52069, 26515, 66383,
         6157, 12231, 25447,  3983, 12264, 35996, 25233, 10825, 67167,
        57733,   653, 13176, 10234, 11628, 52509, 10209, 42193,  8707,
        47738, 33972, 55986, 29512,  3177, 52515, 72527, 26816, 54853,
        54255, 24635, 10410, 42205, 14688,  3355, 11305, 18797, 44809,
        38718, 33376, 72911, 11455, 40795,  2709, 31749, 54415,  8227,
        39315, 41976, 11992, 58645, 67276, 36134, 21969, 60419, 49616,
        32630, 16735,  7260, 32336, 19818,  9562, 66374, 26693, 65703,
        45035, 72710, 37846, 67862, 26659, 70218, 51099, 53066, 49934,
        47918, 61065,  6138, 34558, 44150, 65629,  7539, 53193, 39920,
        56972, 36996, 28118, 37542, 61804, 25032, 34826, 22790,   884,
         6412, 32087, 72373,  9384, 61417, 17373, 20477, 43501, 19425,
        18156, 17843,  2307, 72002, 53819, 50259, 67762, 10674, 32988,
         1233, 53163

In [2]:
import wandb
import seaborn as sns
import numpy as np
from collections import defaultdict
api = wandb.Api()

ENTITY = "anhhuyalex"
PROJECT = "eye_int"
# Filter runs by group name
# runs = api.runs(f"{ENTITY}/{PROJECT}", filters={"group": "jan22_2221_pruning_500"})
runs = api.runs(f"{ENTITY}/{PROJECT}", filters={"group": "jan22_2221_pruning_6000_from_3500"})
 
 
# Download specific data from each run
num_samples = []
left_out = []
test_bwd_pct_correct = defaultdict(list)
test_fwd_pct_correct = defaultdict(list)
test_both_pct_correct = defaultdict(list)
num_runs = 0
for run in runs:
    # run.delete()
#     if run.state == "finished" and run.config.get("num_samples", None) == 6999:
    
    try:
        print (run.summary["_step"])
        if (run.summary["_step"] != 14): continue
    except:
        continue 
    # print ("run.config", run.config)
    offline_logfile = run.config["offline_logfile"] 
    # if  [int(i) for i in offline_logfile[1:100].split(" ") if i != ""] != [70420, 2598, 63069, 32343, 37942, 25934, 32988, 15835, 50918, 48882, 28161, 13467, 6373, 1247, 41268, 66508, 31]: continue
    # num_runs += 1
   

    print ("offline_logfile",offline_logfile)
    
    # display (run.history())#["filter_samples"].iloc[0])
    max_bwd = run.history()["test/test_bwd_pct_correct"].max()
    max_fwd = run.history()["test/test_fwd_pct_correct"].max()
    test_bwd_pct_correct[offline_logfile].append(max_bwd)
    test_fwd_pct_correct[offline_logfile].append(max_fwd)
    test_both_pct_correct[offline_logfile].append((max_bwd+max_fwd)/2)
    
  
    try:
        test_bwd_pct_correct.append( run.history().iloc[19]["test/test_bwd_pct_correct"])
        test_fwd_pct_correct.append( run.history().iloc[19]["test/test_fwd_pct_correct"])
    except:
        continue
mean_test_bwd_pct_correct = {k: np.mean(v) for k,v in test_bwd_pct_correct.items()} 
mean_test_fwd_pct_correct = {k: np.mean(v) for k,v in test_fwd_pct_correct.items()}
mean_test_both_pct_correct = {k: np.mean(v) for k,v in test_both_pct_correct.items()}
best_test_bwd_pct_correct = sorted(mean_test_bwd_pct_correct.items(), key=lambda x: x[1], reverse=True) 
best_test_fwd_pct_correct = sorted(mean_test_fwd_pct_correct.items(), key=lambda x: x[1], reverse=True)
best_test_both_pct_correct = sorted(mean_test_both_pct_correct.items(), key=lambda x: x[1], reverse=True)
print ("best_test_fwd_pct_correct", best_test_fwd_pct_correct) 
print ("best_test_bwd_pct_correct", best_test_bwd_pct_correct)
print ("best_test_both_pct_correct", best_test_both_pct_correct)
# sns.set_theme(style="whitegrid")
# ax = sns.histplot(  x=test_fwd_pct_correct,   label="test_fwd_pct_correct")
# ax = sns.histplot( x=test_bwd_pct_correct,  label="test_bwd_pct_correct")


# ax.legend(loc="lower right")


14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_71.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_76.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_86.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_51.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_7.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_49.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_83.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_75.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_70.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_73.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_

14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_86.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_51.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_83.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_70.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_75.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_49.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_79.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_7.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_73.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_81.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_

14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_73.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_70.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_72.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_82.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_51.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_91.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_79.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_77.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_83.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_81.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000

14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_79.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_81.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_87.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_82.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_72.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_70.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_85.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_66.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_83.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_77.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000

14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_83.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_84.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_81.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_80.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_78.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_79.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_77.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_76.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_75.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000_from_3500_seed_72.log
14
offline_logfile ./activelearning_cache/finetune_numsampleskeys_6000

In [5]:
import copy
import pickle 
print( [len(v) for k,v in test_bwd_pct_correct.items()] )
print([ len(v) for k,v in test_fwd_pct_correct.items()] )
print([  (v) for k,v in best_test_both_pct_correct ] )
best_filter_list, best_both = best_test_both_pct_correct[0]

print ("best_filter_list, best_both",best_filter_list, best_both, len(best_test_both_pct_correct))
 
with open(best_filter_list, "rb") as f:
    best_filter_samples = pickle.load(f)
best_filter_samples = best_filter_samples["filter_samples"]
print ("best_filter_samples", len(set(best_filter_samples)), best_filter_samples[:100])

other_keys = [k for k in keys if k not in best_filter_samples]
# verify that the keys are unique
print ("keys", len(keys), len(other_keys)) 
assert not any(x in set(other_keys) for x in best_filter_samples), "Found an element of best_filter_samples in other_keys"
num_keys_chosen = 500
for seed in range(100):
    np.random.shuffle(other_keys) 
    other_keys_500 = copy.deepcopy(other_keys[:num_keys_chosen]) 
    best_filter_samples_and_other_keys = np.concatenate([best_filter_samples, other_keys_500] )
    print(best_filter_samples_and_other_keys[5995:6005], best_filter_samples_and_other_keys[-10:], len(best_filter_samples_and_other_keys))
    np.save (f"./cache/keys_{len(set(best_filter_samples))+num_keys_chosen}_from_{9000-len(set(best_filter_samples))}_seed_{seed}.npy", best_filter_samples_and_other_keys)

[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 6, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 6, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
[0.8256666958332062, 0.8253333508968353, 0.821333360671997, 0.8210000216960907, 0.8196666896343231, 0.8190000176429748, 0.8186666905879975, 0.8183333396911621, 0.8180000185966492, 0.8180000126361847, 0.8170000195503235, 0.8166666746139526, 0.8163333594799042, 0.8160000264644622, 0.8146666824817658, 0.8143333494663239, 0.8143333435058594, 0.8143333435058594, 0.8140000104904175, 0.8140000104904175, 0.81366668343544, 0.81366668343544, 0.8136666774749756, 0.8136666715145111, 

[62047 18208 38834 42249 53899 58930 32541 69137 41202 55685] [31735  4750 24943 15685 53144 32531 31655 46943  7940 45548] 6500
[62047 18208 38834 42249 53899 43622   608 53396 35512  9296] [21048 25566  1743 28835 41536 43626 16367 45588 30311 24289] 6500
[62047 18208 38834 42249 53899 21107  4033  9408 43205 59565] [25916 34799 16259 55084 43457  2710 65803  8983 72606 64888] 6500
[62047 18208 38834 42249 53899 31724 21643 10054 19593 40996] [49510 51453 55248 15384 72002 35512 21447 52377  1233 31298] 6500
[62047 18208 38834 42249 53899 33319 53684 46613 14429 25029] [13102 66298  7316 47218 12085 53729 64647 33017 36432 65532] 6500
[62047 18208 38834 42249 53899 59006  7502 16743 40795 63283] [64195  7355 24216 40819 37083  2613 28280 65263  7919 56933] 6500
[62047 18208 38834 42249 53899 31982 11455 47121 15738 63739] [16310 58019  2735  8994 24635  5361 53729  1913  4470 46992] 6500
[62047 18208 38834 42249 53899 66776 48740 30213 46943 56750] [19260 44737  4439 66383 11490 3084

# Forward pruning

In [9]:
import wandb
import seaborn as sns
import numpy as np
from collections import defaultdict
api = wandb.Api()

ENTITY = "anhhuyalex"
PROJECT = "eye_int"
# Filter runs by group name
# runs = api.runs(f"{ENTITY}/{PROJECT}", filters={"group": "jan22_2221_pruning_500"})
runs = api.runs(f"{ENTITY}/{PROJECT}", filters={"group": "forwardprune_feb13_adding_100_from_0"})
 
 
# Download specific data from each run
num_samples = []
left_out = []
val_bwd_pct_correct = defaultdict(list)
val_fwd_pct_correct = defaultdict(list)
val_both_pct_correct = defaultdict(list)
num_runs = 0
for run in runs:
    # run.delete()
#     if run.state == "finished" and run.config.get("num_samples", None) == 6999:
    
    try:
        print (run.summary["_step"])
        if (run.summary["_step"] != 14): continue
    except:
        continue 
    # print ("run.config", run.config)
    offline_logfile = run.config["offline_logfile"] 
    # if  [int(i) for i in offline_logfile[1:100].split(" ") if i != ""] != [70420, 2598, 63069, 32343, 37942, 25934, 32988, 15835, 50918, 48882, 28161, 13467, 6373, 1247, 41268, 66508, 31]: continue
    # num_runs += 1
   

    print ("offline_logfile",offline_logfile)
    
    # display (run.history())#["filter_samples"].iloc[0])
    max_bwd = run.history()["val/val_bwd_pct_correct"].max()
    max_fwd = run.history()["val/val_fwd_pct_correct"].max()
    test_bwd_pct_correct[offline_logfile].append(max_bwd)
    test_fwd_pct_correct[offline_logfile].append(max_fwd)
    test_both_pct_correct[offline_logfile].append((max_bwd+max_fwd)/2)
    
  
    try:
        test_bwd_pct_correct.append( run.history().iloc[19]["test/test_bwd_pct_correct"])
        test_fwd_pct_correct.append( run.history().iloc[19]["test/test_fwd_pct_correct"])
    except:
        continue
mean_test_bwd_pct_correct = {k: np.mean(v) for k,v in test_bwd_pct_correct.items()} 
mean_test_fwd_pct_correct = {k: np.mean(v) for k,v in test_fwd_pct_correct.items()}
mean_test_both_pct_correct = {k: np.mean(v) for k,v in test_both_pct_correct.items()}
best_test_bwd_pct_correct = sorted(mean_test_bwd_pct_correct.items(), key=lambda x: x[1], reverse=True) 
best_test_fwd_pct_correct = sorted(mean_test_fwd_pct_correct.items(), key=lambda x: x[1], reverse=True)
best_test_both_pct_correct = sorted(mean_test_both_pct_correct.items(), key=lambda x: x[1], reverse=True)
print ("best_test_fwd_pct_correct", best_test_fwd_pct_correct) 
print ("best_test_bwd_pct_correct", best_test_bwd_pct_correct)
print ("best_test_both_pct_correct", best_test_both_pct_correct)
# sns.set_theme(style="whitegrid")
# ax = sns.histplot(  x=test_fwd_pct_correct,   label="test_fwd_pct_correct")
# ax = sns.histplot( x=test_bwd_pct_correct,  label="test_bwd_pct_correct")


# ax.legend(loc="lower right")


14
offline_logfile ./forward_pruning_cache/finetune_numsamplesforwardprune_keys_6000_from_6500_seed_99.log
14
offline_logfile ./forward_pruning_cache/finetune_numsamplesforwardprune_keys_6000_from_6500_seed_92.log
14
offline_logfile ./forward_pruning_cache/finetune_numsamplesforwardprune_keys_6000_from_6500_seed_33.log
14
offline_logfile ./forward_pruning_cache/finetune_numsamplesforwardprune_keys_6000_from_6500_seed_90.log
14
offline_logfile ./forward_pruning_cache/finetune_numsamplesforwardprune_keys_6000_from_6500_seed_87.log
14
offline_logfile ./forward_pruning_cache/finetune_numsamplesforwardprune_keys_6000_from_6500_seed_77.log
14
offline_logfile ./forward_pruning_cache/finetune_numsamplesforwardprune_keys_6000_from_6500_seed_84.log
14
offline_logfile ./forward_pruning_cache/finetune_numsamplesforwardprune_keys_6000_from_6500_seed_8.log
14
offline_logfile ./forward_pruning_cache/finetune_numsamplesforwardprune_keys_6000_from_6500_seed_86.log
14
offline_logfile ./forward_pruning_c

In [12]:
import copy
import pickle 
print( [len(v) for k,v in test_bwd_pct_correct.items()] )
print([ len(v) for k,v in test_fwd_pct_correct.items()] )
print([  (v) for k,v in best_test_both_pct_correct ] )
best_filter_list, best_both = best_test_both_pct_correct[0]

print ("best_filter_list, best_both",best_filter_list, best_both, len(best_test_both_pct_correct))
 
with open(best_filter_list, "rb") as f:
    best_filter_samples = pickle.load(f)
best_filter_samples = best_filter_samples["filter_samples"]
print ("best_filter_samples", len(set(best_filter_samples)), best_filter_samples[:100])

num_keys_chosen = 500
for seed in range(100):
    np.random.shuffle(best_filter_samples) 
    print(best_filter_samples[:(len(set(best_filter_samples)) - num_keys_chosen)] [-10:], len(best_filter_samples[:(len(set(best_filter_samples)) - num_keys_chosen)]))
    np.save (f"./cache/forwardprune_keys_{len(set(best_filter_samples))-num_keys_chosen}_from_{len(set(best_filter_samples))}_seed_{seed}.npy", best_filter_samples[:(len(set(best_filter_samples)) - num_keys_chosen)])

[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4]
[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4]
[0.8333333551883697, 0.8333333551883697, 0.8310000181198121, 0.8303333640098571, 0.8303333580493927, 0.8293333530426026, 0.8276666879653931, 0.8276666879653931, 0.8276666760444641, 0.8273333549499512, 0.8270000159740448, 0.8266666889190674, 0.8266666769981384, 0.8263333559036254, 0.8256666898727417, 0.8256666779518127, 0.8253333568572998, 0.824666690826416, 0.824666690826416, 0.8246666789054871, 0.824333351850509

# Plot results of active learning search

In [ ]:
import wandb
import seaborn as sns
import numpy as np
from collections import defaultdict
import pandas as pd
import matplotlib.pyplot as plt
api = wandb.Api()

ENTITY = "anhhuyalex"
PROJECT = "eye_int"
# Filter runs by group name
# runs = api.runs(f"{ENTITY}/{PROJECT}", filters={"group": "jan22_2221_pruning_500"})
def load_runs(group_name, num_samples, run_type):
    runs = api.runs(f"{ENTITY}/{PROJECT}", filters={"group": group_name})
    
    
    # Download specific data from each run 
    test_bwd_pct_correct = []
    test_fwd_pct_correct = []
    test_both_pct_correct = []
    num_sample_list = []
    run_type_list = []
    for i, run in enumerate(runs):
     #     if run.state == "finished" and run.config.get("num_samples", None) == 6999:
        
        try:
            print (run.summary["_step"]) 
            
            if (run.summary["_step"] != 14): continue
#             if (len(run.config["samples_list"])== 41): continue
#             print (run.config["samples_list"], len(run.config["samples_list"]))
        except:
            continue 
        if group_name == "jan22_2221_pruning_1000":
            samples_list = run.config["samples_list"]
            print (samples_list[1:100])
            if [int(i) for i in samples_list[1:100].split(" ") if i != ""] != [70420, 2598, 63069, 32343, 37942, 25934, 32988, 15835, 50918, 48882, 28161, 13467, 6373, 1247, 41268, 66508, 31]: 
                run_type = "random"
            else:
                run_type = "backward"
                
            
        # print ("run.config", run.config)
        # offline_logfile = run.config["offline_logfile"] 
        # try: 
        #     print (len( run.config["samples_list"] ))
        # except:
        #     pass 
        # if  [int(i) for i in offline_logfile[1:100].split(" ") if i != ""] != [70420, 2598, 63069, 32343, 37942, 25934, 32988, 15835, 50918, 48882, 28161, 13467, 6373, 1247, 41268, 66508, 31]: continue
        # num_runs += 1
    

        # print ("offline_logfile",offline_logfile)
        
#         display (run.summary['test/test_fwd_pct_correct'] )#["filter_samples"].iloc[0])
        max_bwd = run.history()["test/test_bwd_pct_correct"].max()
        max_fwd = run.history()["test/test_fwd_pct_correct"].max()
        # max_bwd = run.history().iloc[14]["test/test_bwd_pct_correct"]
        # max_fwd = run.history().iloc[14]["test/test_fwd_pct_correct"]
        test_bwd_pct_correct.append(max_bwd)
        test_fwd_pct_correct.append(max_fwd)
        test_both_pct_correct.append((max_bwd+max_fwd)/2)
        num_sample_list.append(num_samples)
        run_type_list.append(run_type)
    
        # try:
        #     test_bwd_pct_correct.append( run.history().iloc[19]["test/test_bwd_pct_correct"])
        #     test_fwd_pct_correct.append( run.history().iloc[19]["test/test_fwd_pct_correct"])
        # except:
        #     continue
    return test_bwd_pct_correct, test_fwd_pct_correct, test_both_pct_correct, num_sample_list, run_type_list
dfs = []
group_name_lists = ["jan22_2221_pruning_500", "jan22_2221_pruning_1000", 
                    "jan22_2221_pruning_1500_from_8000", "jan22_2221_pruning_1500_from_9000",
                    "jan22_2221_pruning_2000_from_7500", "jan22_2221_pruning_2000_from_9000",
                    "jan22_2221_pruning_2500_from_7000", "jan22_2221_pruning_2500_from_9000",
                    "jan22_2221_pruning_3000_from_6500", "jan22_2221_pruning_3000_from_9000",
                    "jan22_2221_pruning_3500_from_6000", "jan22_2221_pruning_3500_from_9000",
                      "jan22_2221_pruning_4000_from_5500", "jan22_2221_pruning_4000_from_9000",
                    "jan22_2221_pruning_4500_from_5000", "jan22_2221_pruning_4500_from_9000",
                    "jan22_2221_pruning_5000_from_4500", "jan22_2221_pruning_5000_from_9000",
                    "jan22_2221_pruning_5500_from_4000", "jan22_2221_pruning_5500_from_9000",
                    "jan22_2221_pruning_6000_from_3500", "jan22_2221_pruning_6000_from_9000",
                                                         "jan22_2221_pruning_6500_from_9000",
                                                         "jan22_2221_pruning_7000_from_9000",
                                                         "jan22_2221_pruning_7500_from_9000",
                                                         "jan22_2221_pruning_8000_from_9000",
                                                         "jan22_2221_pruning_8500_from_9000",
                    "forwardprune_jan22_2221_pruning_6000_from_6500",
                    "forwardprune_jan22_2221_pruning_6500_from_7000",
                    "forwardprune_jan22_2221_pruning_7000_from_7500",
                    "forwardprune_jan22_2221_pruning_7500_from_8000",
                    "forwardprune_jan22_2221_pruning_8000_from_8500",
                   ]
num_samples_lists = [8500, 8000,
                    7500, 7500,
                    7000, 7000,
                    6500, 6500,
                    6000, 6000,
                    5500, 5500,
                    5000, 5000,
                    4500, 4500,
                    4000, 4000,
                    3500, 3500,
                    3000, 3000,
                          2500,
                          2000,
                          1500,
                          1000,
                          500,
                     3000,
                     2500,
                     2000,
                     1500,
                     1000
                    ]
run_type_lists = ["random", "random",
                  "backward", "random",
                  "backward", "random",
                  "backward", "random",
                  "backward", "random",
                  "backward", "random",
                  "backward", "random",
                  "backward", "random",
                  "backward", "random",
                  "backward", "random",
                  "backward", "random",
                              "random",
                              "random",
                              "random",
                              "random",
                              "random",
                  "forward",
                  "forward",
                  "forward",
                  "forward",
                  "forward"
                 ]
for (group_name, num_samples, run_type) in zip(group_name_lists, num_samples_lists, run_type_lists):
    test_bwd_pct_correct, test_fwd_pct_correct, test_both_pct_correct, num_sample_list, run_type_list = load_runs(group_name, num_samples, run_type)
    df = pd.DataFrame({"test_bwd_pct_correct": test_bwd_pct_correct, "test_fwd_pct_correct": test_fwd_pct_correct, "test_both_pct_correct": test_both_pct_correct, "num_samples": num_sample_list,
                    "run_type_list": run_type_list})
    dfs.append(df)

dfs = pd.concat(dfs)
x

14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
0
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
4
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
5
14
14
14
5
14
14
5
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
3
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
4
14
14
4
4
14
14
14
14
14
14
4
14
14
14
4
14
4
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
1

14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 1346

14
14765  8367 59586 65031 26745 44879 49577 47943  6332  8286 65352 34137
 52228 20496 46281 30415 25
14
14940 22900 44173  4691 18900 54907 65008 21325 41082 59975 27813 45413
 69177 55847 61540 50289 39
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 1346

14
25725 47509 62949 64208 61082  3715 35394  6938 16532 21609 45936  3755
 63034 68699 72122  2999 34
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 1346

14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
14940 22900 44173  4691 18900 54907 65008 21325 41082 59975 27813 45413
 69177 55847 61540 50289 39
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 1346

...

14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 1346

14
53037 12386 47700 38236 13849 15573 15977  7651 42445 66866 58171 60104
 64850 46443 13102 60858 53
14
32071 68166  8426 62749 30925 22923 23984 60159 57814 49616 24304 12725
  7699 37769 27901 33585 42
14
 5696 22233 41203 64169 50644  1273 31503 71701  6657  5830 51570 15510
 13682 39177 31714  6752 28
14
47128  5165   992 64677 23595 72710 27131 38338 28473 18736 62393 24748
 25322  7264 26277 28895 41
14
70074 18334 61414  3301 38906 29815  8862 26735 47561 10632  3811  9255
 45140 38663 19653 21570  4
14
12466 35275 21945 70706 26861  4729 23527 43968 15143 23865  9712 23401
 69318 20562 39745 51251 33
14
68355 12593 28925  5558 62741 53540  3182 30013 26070 31262 65966 22236
 57953 24390 61875 28454 68
14
57994 13134  4343 16735 50200 29881 42538  3276 44378 36834 28084 63852
 57550  6876 72645  5918 36
14
52484 15541 26504 70327 38634 45699  5121 65387 31905 41540  1396 56225
 57119 35354 27685 40061  6
14
21054 13986 51830 10255 36814 34826 37920 56873 66520 21520 51215 1046

14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 13467
  6373  1247 41268 66508 31
14
70420  2598 63069 32343 37942 25934 32988 15835 50918 48882 28161 1346

14
57994 13134  4343 16735 50200 29881 42538  3276 44378 36834 28084 63852
 57550  6876 72645  5918 36
14
21937 41377  3925 38670  8983 52767 54049 22381 19384 65778 61171 61344
 46153 70960 72095  2264 69
14
68355 12593 28925  5558 62741 53540  3182 30013 26070 31262 65966 22236
 57953 24390 61875 28454 68
14
65993 65339 20951 42052 68166  8389 55638 17636 16727 23739  6044 29492
 10355 36841 49375 22120  6
14
29833 11018 37621 42945 58279 55878 66512 27976 57119 11655 70569 46683
 60302 51697 30748 29349 15
14
52484 15541 26504 70327 38634 45699  5121 65387 31905 41540  1396 56225
 57119 35354 27685 40061  6
14
55172 51913  5756 37680 42052 43812 20013 32931 67428 50259 28929 47042
 49748 28003 35832 70833 58
14
57729 26478 18648 14135 35176 71330 14920 32464 56951 52971 12229 19989
 59093 37998  4711  3893 15
14
55641 58046 33232 16962  1521  2962  8304 22237 61589 13289 25960 10459
 28525 20902 62054 34457 14
14
40881 68391 61461 10445 11551 24845 51830 40977 72226  4362 33866  823

14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
1

In [ ]:
pd.set_option('display.max_rows', 500)
# display(dfs)
sns.set_theme(style="whitegrid")
ax = sns.lineplot( x="num_samples", y="test_both_pct_correct", marker="o", hue="run_type_list", data=dfs,errorbar=lambda x:  (np.min(x),np.max(x)))
plt.show()

In [ ]:
dfs.to_csv("result_of_active_learning.csv")

# Kmeans data selection with fMRI

In [1]:
import webdataset as wds 
import random
import torch
import h5py
import numpy as np
import matplotlib.pyplot as plt
from qpsolvers import solve_qp

def my_split_by_node(urls): return urls

# f = h5py.File(f'/weka/proj-fmri/shared/mindeyev2_dataset/betas_all_subj01_fp32.hdf5', 'r')
f = h5py.File(f'/weka/home-alexnguyen/mindeyev2_dataset/betas_all_subj01_fp32.hdf5', 'r')
voxels = f['betas'][:]
del f 
print (voxels.shape)
# train_url = "/weka/proj-fmri/shared/mindeyev2_dataset/wds/subj01/new_train/{0..39}.tar"
train_url = "/weka/home-alexnguyen/mindeyev2_dataset/wds/subj01/new_train/{0..39}.tar"
train_data = wds.WebDataset(train_url,resampled=True,nodesplitter=my_split_by_node)\
                        .shuffle(750, initial=2500, rng=random.Random(1))\
                        .decode("torch")\
                       .rename(behav="behav.npy", past_behav="past_behav.npy", future_behav="future_behav.npy", olds_behav="olds_behav.npy")\
                       .to_tuple(*["__key__", "behav", "past_behav", "future_behav", "olds_behav"]) 
train_dl = torch.utils.data.DataLoader(train_data, batch_size=512, shuffle=False, drop_last=True, pin_memory=True) 
keys = set() 
for iter, (key, behav0, past_behav0, future_behav0, old_behav0) in enumerate(train_dl): 
    
#     print (behav0[:,0,0].cpu().long())
#     print (past_behav0[:,0,0].cpu().long())
    [keys.add( i.item() ) for i in behav0[:,0,5].cpu().long()]
#     print ((keys))
    if iter % 10 == 1: print (len(keys))
    if len(keys) >= 27000: break

(30000, 15724)
1024
5839
9172
11673
14837
17539
18221
19379
22140
22471
23621
24298
25587
25639
25639
26328
26329
26329
26329
26329
26329
26329
26329
26329
26329
26998


## Make random 

In [7]:
seed=10
keys = list(keys)

for N in [750, 1500, 2250, 3000, 6000, 9000, 12000, 15000, 18000, 21000, 24000, 27000]:
    random_ids = np.random.choice(keys, N, replace=False)
    np.save(f"./cache/random_fmriraw_kmeans_pruned_ids_N_{int(N)}_seed_{seed}.npy", random_ids)
    print ("N", random_ids[:100], len(random_ids), len(np.unique(random_ids)))
    # plt.scatter(d_intra,d_inter)
    # plt.show()
    # plt.plot(Pj) 



N [ 5309  5729 14770 11825  2426  4137  8230  2789  6411 24433 14815 18284
 10213 13442 22247  5675 27967   935  2932 21106  1580 15314 23039  6372
 25148 21863 19124 29266   321 25083 19417  5671 21020  4411  5238 29775
 29397   713 16179  7678  5724  2688 18589 10941 11644  5672 27366   611
 13622   839 13341 22156 17673  8579 14863 17686 28207 12641 20293 27406
  8401 14754 13886  2866 24330 23691 27692 10759 21521  9060 15882 16791
 20494   259 21306 29996 15165 20970 29296    57 21050 25311 10034 17342
   977  2387   994  4302 22234 17948 24481 21635 23846 15045 12077  9958
 21564 28224 12593  4534] 750 750
N [27591 24009 13752 25680 14884 12334  3860  8999 28566 10492  7124  7403
 20808  1922 28158  1031 28609 16218  6510 28769 26273 28577  7142 12456
 29080   899  6975 26098 11334  6220 26118 13581 26507  4957 21574  1530
   134 27335 20873 25496  9655 15336 14993  2112 13161 14892  2521 15823
 29043 17602 28455  8758  2321  7998 13125   765  8698  9781 18069  5366
 14092  3376 

## Clustering

In [2]:
# get voxels using keys
keys = list(keys)
_voxels = voxels[keys]
# convert to numpy
print (_voxels.shape)

(27000, 15724)


In [10]:
from sklearn.cluster import KMeans
n_clusters = 100
seed=4
kmeans = KMeans(n_clusters=n_clusters, random_state=seed, n_init="auto",verbose=2).fit(_voxels)
kmeans.labels_, kmeans.labels_.shape

Initialization complete
Iteration 0, inertia 584423936.0.
Iteration 1, inertia 358799904.0.
Iteration 2, inertia 354673472.0.
Iteration 3, inertia 353362304.0.
Iteration 4, inertia 352646272.0.
Iteration 5, inertia 352191616.0.
Iteration 6, inertia 351904288.0.
Iteration 7, inertia 351690784.0.
Iteration 8, inertia 351534048.0.
Iteration 9, inertia 351425152.0.
Iteration 10, inertia 351350560.0.
Iteration 11, inertia 351292160.0.
Iteration 12, inertia 351246688.0.
Iteration 13, inertia 351209568.0.
Iteration 14, inertia 351182784.0.
Iteration 15, inertia 351165728.0.
Iteration 16, inertia 351151360.0.
Iteration 17, inertia 351139040.0.
Iteration 18, inertia 351128576.0.
Iteration 19, inertia 351119648.0.
Iteration 20, inertia 351111904.0.
Iteration 21, inertia 351105888.0.
Iteration 22, inertia 351100608.0.
Iteration 23, inertia 351094912.0.
Iteration 24, inertia 351089504.0.
Iteration 25, inertia 351084192.0.
Iteration 26, inertia 351079840.0.
Iteration 27, inertia 351076448.0.
Iterat

(array([69, 93, 11, ..., 48, 44, 82], dtype=int32), (27000,))

In [11]:
# Calculate dintra and dinter for each cluster. 
# dintra is computed as the average cosine distance between the items of a cluster and its centroid
# dinter is computed for each cluster j as the average cosine distance between a cluster centroid and its l nearest
# neighbor centroids
def compute_dintra_dinter(kmeans, n_neighbors = 20, n_clusters = 100): 
    """
    kmeans: kmeans object  
    n_neighbors: number of neighbors to use for computing dinter
    n_clusters: number of clusters
    """
    d_intra = np.zeros(n_clusters)
    d_inter = np.zeros(n_clusters)
    
    for cluster_id in range(n_clusters):
        cluster = _voxels[kmeans.labels_ == cluster_id]
        # d_intra[cluster_id] = (np.linalg.norm(cluster - kmeans.cluster_centers_[cluster_id], axis=1).mean())
        a_dot_b = np.dot(cluster, np.expand_dims(kmeans.cluster_centers_[cluster_id], axis=1))
        a_norm = np.linalg.norm(cluster, axis=1) 
        b_norm = np.linalg.norm(kmeans.cluster_centers_[cluster_id])
        # print (a_dot_b.shape, a_norm.shape, b_norm, cluster.shape, np.expand_dims(kmeans.cluster_centers_[cluster_id], axis=1).shape)
        d_intra[cluster_id] = 1-(a_dot_b.squeeze(1) / (a_norm * b_norm)).mean()

        # compute distance from this cluster to all other clusters
        a_dot_b = np.dot(np.expand_dims(kmeans.cluster_centers_[cluster_id], axis=0), kmeans.cluster_centers_.T)
        a_norm = np.linalg.norm(np.expand_dims(kmeans.cluster_centers_[cluster_id], axis=0), axis=1)
        b_norm = np.linalg.norm(kmeans.cluster_centers_, axis=1)
        # print (kmeans.cluster_centers_[cluster_id].shape, kmeans.cluster_centers_.T.shape, a_dot_b.shape, a_norm.shape, b_norm.shape)
        distance_to_other_clusters = 1-(a_dot_b / (a_norm * b_norm))  
        d_inter[cluster_id] = np.sort(distance_to_other_clusters)[0,1:(n_neighbors+1)].mean() # 0 is the distance to itself

    return d_intra, d_inter

def pruning(kmeans, d_intra, d_inter, n_clusters = 100, tau = 0.1, fraction = 0.5, N = 27000): 
    """
    kmeans: kmeans object  
    d_intra: dintra for each cluster
    d_inter: dinter for each cluster
    n_clusters: number of clusters
    tau: temperature for softmax
    fraction: fraction of the dataset to use for computing Nj
    N: target dataset size
    """
    # print ("distance_to_other_clusters", distance_to_other_clusters, "distance_closest_neighbors", distance_closest_neighbors) 
    # Calculate the number of examples per cluster Nj. 
    Cj = d_intra * d_inter
    # softmax
    Pj = np.exp(Cj/tau) / np.exp(Cj/tau).sum()
    Nj = Pj * N 

    # solve quadratic program to fix Nj
    probs = torch.softmax(torch.tensor(Cj/tau),dim=0)
    # print(probs,probs.shape,Pj)
    P = np.eye(n_clusters)
    q = - Nj
    A = np.array([1.0] * n_clusters)
    b = np.array([N])
    # Define the lower and upper bounds
    min_samples = 1
    num_items_in_each_cluster = np.bincount(kmeans.labels_)
    bounds = np.array([ ( min_samples, num_items_in_each_cluster[i] ) for i in range(n_clusters) ])
    # print (bounds.shape,bounds[:,[0]].shape,bounds[:,[1]].shape,P.shape, q.shape)
    X = solve_qp(P=P, q=q, A=A, b=b, lb=bounds[:,0].reshape(1,n_clusters), ub=bounds[:,1].reshape(1,n_clusters), solver="osqp")
    Nj = np.rint(X).astype(int)
    # print(Nj)
    pruned_ids = []
    for cluster_id in range(n_clusters):
        
        cluster_ids = np.where( kmeans.labels_ == cluster_id)[0]
        
        cluster = _voxels[cluster_ids]
        nsamples_cluster = Nj[cluster_id]
        # compute cosine similarity of cluster to cluster_centers_[cluster_id]
        a_dot_b = np.dot(cluster, np.expand_dims(kmeans.cluster_centers_[cluster_id], axis=1))
        a_norm = np.linalg.norm(cluster, axis=1)
        b_norm = np.linalg.norm(kmeans.cluster_centers_[cluster_id])
        distance_to_centroid = 1-(a_dot_b.squeeze(1) / (a_norm * b_norm))
        distance_to_centroid = np.argsort(distance_to_centroid)[-nsamples_cluster:]
        pruned_ids.extend([keys[i] for i in cluster_ids[distance_to_centroid]]) 
        
        # select nsamples_cluster from cluster that are furthest away from cluster centroid
        # print (cluster.shape, distance_to_centroid.shape)

    return pruned_ids, Nj, Pj
d_intra, d_inter = compute_dintra_dinter(kmeans)
for N in [750, 1500, 2250, 3000, 6000, 9000, 12000, 15000, 18000, 21000, 24000, 27000]:
    pruned_ids, Nj, Pj = pruning(kmeans, d_intra, d_inter, N=N)
    np.save(f"./cache/fmriraw_kmeans_pruned_ids_N_{int(N)}_seed_{seed}.npy", pruned_ids)
    print ("N", N, pruned_ids[:100], len(pruned_ids), "seed", seed)
    # plt.scatter(d_intra,d_inter)
    # plt.show()
    # plt.plot(Pj) 



/weka/home-alexnguyen/mind-reader/MindEyeV2/src/mindeye/lib/python3.11/site-packages/qpsolvers/conversions/ensure_sparse_matrices.py:24: UserWarning: Converted P to scipy.sparse.csc.csc_matrix
For best performance, build P as a scipy.sparse.csc_matrix rather than as a numpy.ndarray
  warnings.warn(
/weka/home-alexnguyen/mind-reader/MindEyeV2/src/mindeye/lib/python3.11/site-packages/qpsolvers/conversions/ensure_sparse_matrices.py:24: UserWarning: Converted A to scipy.sparse.csc.csc_matrix
For best performance, build A as a scipy.sparse.csc_matrix rather than as a numpy.ndarray
  warnings.warn(


N 750 [29003, 10739, 24817, 16315, 8742, 4397, 13018, 25135, 18576, 3508, 4506, 9081, 8922, 11295, 11567, 156, 21316, 22351, 28325, 23821, 10979, 21071, 14894, 26993, 27892, 7704, 20113, 24832, 14500, 12371, 5709, 24466, 25823, 18605, 4720, 4737, 4726, 4730, 29366, 28512, 14157, 19464, 28322, 6525, 18787, 25692, 9224, 2157, 16195, 8532, 3659, 15530, 12635, 10851, 10871, 16632, 12960, 12312, 2008, 19714, 20679, 20479, 22045, 21956, 21558, 17150, 9085, 23234, 6341, 4838, 23883, 10781, 3583, 13381, 25861, 9082, 27471, 20031, 25686, 98, 17263, 24665, 29095, 22708, 23508, 21862, 29109, 13441, 10175, 29534, 8608, 13507, 13679, 3780, 8654, 2434, 20242, 5508, 15128, 377] 747 seed 4
N 1500 [29019, 29004, 29003, 25044, 27010, 7567, 6414, 29237, 20808, 10739, 24817, 16315, 8742, 26505, 13003, 12957, 25152, 28587, 7213, 15778, 25958, 4397, 13018, 25135, 18576, 3508, 4506, 9081, 11508, 8922, 16747, 6204, 24658, 15254, 14076, 14217, 11193, 18871, 2499, 10258, 11295, 11567, 156, 21316, 22351, 28325, 

In [4]:
keys = list(keys)
len(keys)

27000

N [21807 29764  5903 11801 19752 25745 24793 27082  2178 23384 17755 10452
 10621  5393 24215 13998 11102  1014 27666 12285 23460 12201 27000  8330
 22299  2342 21872 14049  2781  9452 26901 13752 29405 24189 18495 10178
 22621 13210 23246 17199 26227 15943 15732 16806 23630  8843  7975 12306
 27123 16252 10252 23536  1924 17914  7478  1655 21609 17725   579 15745
 27510 23373 16054 24526 26192 26204 19093 24380  6307  4655  4080 19291
 24959  7803  7823 13549 12758 27990 25596 14062 22169 14862  3673 27063
 21936  7937 23350 29869 25042  8927 26342  3504 29570 24225 15044 15188
 21737  4484 28913  6015] 750 750
N [  457  6167 23091  8376  6836 21379 13819 29354  2643 23580 24565 23739
  9844 21116 13532 17516 24700 14006 21256 19559 26960 22432 15677 25828
 19959 13887 22837  8217 21988  7307 13700 29844 23622 23142 27334 15130
  4857 12093 10471 22120 12685  4132 11723  4287 10284 13489  7856 28094
 14466  3582  4207 21592  6991 12980  6388 21916  1111  7533 21465 19774
 14056  2628 

# Kmeans data selection with CLIP

In [1]:
import webdataset as wds 
import random
import torch
import h5py
import numpy as np
import matplotlib.pyplot as plt
from qpsolvers import solve_qp

def my_split_by_node(urls): return urls
 

f = h5py.File(f'/weka/proj-fmri/shared/mindeyev2_dataset/coco_images_224_float16.hdf5', 'r')
images = f['images'][:]
images = torch.Tensor(images).to("cpu") 
del f 
print (images.shape)

train_url = "/weka/proj-fmri/shared/mindeyev2_dataset/wds/subj01/new_train/{0..39}.tar"
train_data = wds.WebDataset(train_url,resampled=True,nodesplitter=my_split_by_node)\
                        .shuffle(750, initial=2500, rng=random.Random(1))\
                        .decode("torch")\
                       .rename(behav="behav.npy", past_behav="past_behav.npy", future_behav="future_behav.npy", olds_behav="olds_behav.npy")\
                       .to_tuple(*["__key__", "behav", "past_behav", "future_behav", "olds_behav"]) 
train_dl = torch.utils.data.DataLoader(train_data, batch_size=512, shuffle=False, drop_last=True, pin_memory=True) 
keys = set() 
for iter, (key, behav0, past_behav0, future_behav0, old_behav0) in enumerate(train_dl): 
    
#     print (behav0[:,0,0].cpu().long())
#     print (past_behav0[:,0,0].cpu().long())
    [keys.add( i.item() ) for i in behav0[:,0,0].cpu().long()]
#     print ((keys))
    if iter % 10 == 1: print (len(keys))
    if len(keys) >= 9000: break

torch.Size([73000, 3, 224, 224])
835
3747
5390
6327
7449
7807
8163
8368
8532
8641
8805
8875
8902
8992


## Random clips

In [4]:
for seed in range(5,16):
    keys = list(keys)

    for N in [750, 1500, 2250, 3000, 3750, 4500, 5250, 6000]:
        random_ids = np.random.choice(keys, N, replace=False)
        np.save(f"/weka/home-alexnguyen/mind-reader/MindEyeV2/src/cache/random_clipembds_pruned_ids_N_{int(N)}_seed_{seed}.npy", random_ids)
        print ("N", random_ids[:100], len(random_ids), len(np.unique(random_ids)))
        # plt.scatter(d_intra,d_inter)
        # plt.show()
        # plt.plot(Pj) 



N [53536 59729  6441 19977 65001 40729 11616 33242 48045 38262  4409 21345
 12782 58784 65647 62609 70364 55922  8444 63096 19954 61980  4287 43798
  3808 35837   356  5775 32501 67412 56407 68121 62342 41852 61458 45140
 33442 54520 28147 19682 32100 68029 65770 61875 19370 22373 42392 59917
 58392 48413 32139 18028 70949 37542 14274 56435 60993 65517 25085 55808
 40965 38212 36192  4753 69463  4360 69158 43524 43069 37555 67741 69453
 18240 57507 25546 70607 40473  5892 11251 34817 16536 50385  6953 13993
 34064  6838 42945 46943 58156  7390 28669 21649  2459 71774 44776 46596
 67775 21333 67691 29892] 750 750
N [57894 34064 39754 38491 64203 56664 23470 45035 25322 13948 49313 42757
 51113 26958 17296 49212 49748 15510 15733 53042 67099 47902 27993 18312
 11805  7416  2401 68297  5436 45500 32283 23230 24647 69881 28983 36918
 59853 36226 62722 33930 27688 29043 21918 11638 23057 19301  5340 63194
  3034 57740 53652 13993 22497 38911 55125  6877 69219 44634 56869 27278
 28410 53386 

In [2]:
# get voxels using keys
keys = list(keys)
_images = images[keys]
# convert to numpy
print (_images.shape)

torch.Size([9000, 3, 224, 224])


In [3]:
import torch
from diffusers import VersatileDiffusionPipeline, UniPCMultistepScheduler
# if you get an error here, make sure your diffusers package is version 0.13.0!
data_type = torch.float32
cache_dir = "/weka/home-alexnguyen"
device = torch.device("cuda:0")
vd_pipe = VersatileDiffusionPipeline.from_pretrained("shi-labs/versatile-diffusion", torch_dtype=data_type, cache_dir=cache_dir)
vd_pipe.scheduler = UniPCMultistepScheduler.from_pretrained("shi-labs/versatile-diffusion", subfolder="scheduler", cache_dir=cache_dir)
# vd_pipe = VersatileDiffusionPipeline.from_pretrained("/weka/proj-fmri/shared/cache/versatile-diffusion")#, torch_dtype=data_type)
# vd_pipe.scheduler = UniPCMultistepScheduler.from_pretrained("/weka/proj-fmri/shared/cache/versatile-diffusion", subfolder="scheduler")
# vd_pipe = VersatileDiffusionPipeline.from_pretrained("/weka/home-alexnguyen/models--shi-labs--versatile-diffusion")#, torch_dtype=data_type)
# vd_pipe.scheduler = UniPCMultistepScheduler.from_pretrained("/weka/home-alexnguyen/models--shi-labs--versatile-diffusion", subfolder="scheduler")

vd_pipe.to(device)#(torch.device(f"cuda:{local_rank}"))
clip_model = vd_pipe.image_encoder
clip_model.to(data_type)
clip_model.eval()
clip_model.requires_grad_(False)
clip_seq_dim = 257
clip_emb_dim = 768

## testing it out:
# utils.get_clip_embeddings(clip_model,images[:1].to(device))

Fetching 17 files:   0%|          | 0/17 [00:00<?, ?it/s]

/weka/home-alexnguyen/mind-reader/MindEyeV2/src/mindeye/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [4]:
import utils
clip_embds = []
for batch_ids in range(0, 9000, 300):
    img_batch = _images[batch_ids:batch_ids+300].to(device)
    clip_embds.append(utils.get_clip_embeddings(clip_model,img_batch))
clip_embds = torch.cat(clip_embds, dim=0)

In [5]:
from sklearn.cluster import KMeans
n_clusters = 100
clip_embds = clip_embds.cpu().numpy().reshape(clip_embds.shape[0],-1)
kmeans = KMeans(n_clusters=n_clusters, random_state=1, n_init="auto",verbose=2).fit(clip_embds)
kmeans.labels_, kmeans.labels_.shape

Initialization complete
Iteration 0, inertia 410685984.0.
Iteration 1, inertia 226060432.0.
Iteration 2, inertia 224771408.0.
Iteration 3, inertia 224296400.0.
Iteration 4, inertia 224063408.0.
Iteration 5, inertia 223893120.0.
Iteration 6, inertia 223759152.0.
Iteration 7, inertia 223666912.0.
Iteration 8, inertia 223602848.0.
Iteration 9, inertia 223555072.0.
Iteration 10, inertia 223521536.0.
Iteration 11, inertia 223495984.0.
Iteration 12, inertia 223482704.0.
Iteration 13, inertia 223475168.0.
Iteration 14, inertia 223472128.0.
Iteration 15, inertia 223470288.0.
Iteration 16, inertia 223469120.0.
Iteration 17, inertia 223468208.0.
Iteration 18, inertia 223467632.0.
Iteration 19, inertia 223467376.0.
Converged at iteration 19: strict convergence.


(array([77, 59, 57, ..., 94, 26,  5], dtype=int32), (9000,))

In [7]:
# Calculate dintra and dinter for each cluster. 
# dintra is computed as the average cosine distance between the items of a cluster and its centroid
# dinter is computed for each cluster j as the average cosine distance between a cluster centroid and its l nearest
# neighbor centroids
def compute_dintra_dinter(kmeans, n_neighbors = 20, n_clusters = 100): 
    """
    kmeans: kmeans object  
    n_neighbors: number of neighbors to use for computing dinter
    n_clusters: number of clusters
    """
    d_intra = np.zeros(n_clusters)
    d_inter = np.zeros(n_clusters)
    
    for cluster_id in range(n_clusters):
        cluster = clip_embds[kmeans.labels_ == cluster_id]
        # d_intra[cluster_id] = (np.linalg.norm(cluster - kmeans.cluster_centers_[cluster_id], axis=1).mean())
        a_dot_b = np.dot(cluster, np.expand_dims(kmeans.cluster_centers_[cluster_id], axis=1))
        a_norm = np.linalg.norm(cluster, axis=1) 
        b_norm = np.linalg.norm(kmeans.cluster_centers_[cluster_id])
        # print (a_dot_b.shape, a_norm.shape, b_norm, cluster.shape, np.expand_dims(kmeans.cluster_centers_[cluster_id], axis=1).shape)
        d_intra[cluster_id] = 1-(a_dot_b.squeeze(1) / (a_norm * b_norm)).mean()

        # compute distance from this cluster to all other clusters
        a_dot_b = np.dot(np.expand_dims(kmeans.cluster_centers_[cluster_id], axis=0), kmeans.cluster_centers_.T)
        a_norm = np.linalg.norm(np.expand_dims(kmeans.cluster_centers_[cluster_id], axis=0), axis=1)
        b_norm = np.linalg.norm(kmeans.cluster_centers_, axis=1)
        # print (kmeans.cluster_centers_[cluster_id].shape, kmeans.cluster_centers_.T.shape, a_dot_b.shape, a_norm.shape, b_norm.shape)
        distance_to_other_clusters = 1-(a_dot_b / (a_norm * b_norm))  
        d_inter[cluster_id] = np.sort(distance_to_other_clusters)[0,1:(n_neighbors+1)].mean() # 0 is the distance to itself

    return d_intra, d_inter

def pruning(kmeans, d_intra, d_inter, n_clusters = 100, tau = 0.1, fraction = 0.5, N = 27000): 
    """
    kmeans: kmeans object  
    d_intra: dintra for each cluster
    d_inter: dinter for each cluster
    n_clusters: number of clusters
    tau: temperature for softmax
    fraction: fraction of the dataset to use for computing Nj
    N: target dataset size
    """
    # print ("distance_to_other_clusters", distance_to_other_clusters, "distance_closest_neighbors", distance_closest_neighbors) 
    # Calculate the number of examples per cluster Nj. 
    Cj = d_intra * d_inter
    # softmax
    Pj = np.exp(Cj/tau) / np.exp(Cj/tau).sum()
    Nj = Pj * N 

    # solve quadratic program to fix Nj
    probs = torch.softmax(torch.tensor(Cj/tau),dim=0)
    # print(probs,probs.shape,Pj)
    P = np.eye(n_clusters)
    q = - Nj
    A = np.array([1.0] * n_clusters)
    b = np.array([N])
    # Define the lower and upper bounds
    min_samples = 1
    num_items_in_each_cluster = np.bincount(kmeans.labels_)
    bounds = np.array([ ( min_samples, num_items_in_each_cluster[i] ) for i in range(n_clusters) ])
    # print (bounds.shape,bounds[:,[0]].shape,bounds[:,[1]].shape,P.shape, q.shape)
    X = solve_qp(P=P, q=q, A=A, b=b, lb=bounds[:,0].reshape(1,n_clusters), ub=bounds[:,1].reshape(1,n_clusters), solver="osqp")
    Nj = np.rint(X).astype(int)
    # print(Nj)
    print(sum(Nj), Nj)
    pruned_ids = []
    for cluster_id in range(n_clusters):
        
        cluster_ids = np.where( kmeans.labels_ == cluster_id)[0]
        # print("cluster_ids", cluster_id, cluster_ids)
        cluster = clip_embds[cluster_ids]
        nsamples_cluster = Nj[cluster_id]
        # compute cosine similarity of cluster to cluster_centers_[cluster_id]
        a_dot_b = np.dot(cluster, np.expand_dims(kmeans.cluster_centers_[cluster_id], axis=1))
        a_norm = np.linalg.norm(cluster, axis=1)
        b_norm = np.linalg.norm(kmeans.cluster_centers_[cluster_id])
        distance_to_centroid = 1-(a_dot_b.squeeze(1) / (a_norm * b_norm))
        distance_to_centroid = np.argsort(distance_to_centroid)[-nsamples_cluster:]
        pruned_ids.extend([keys[i] for i in cluster_ids[distance_to_centroid]]) 
        # select nsamples_cluster from cluster that are furthest away from cluster centroid
        # print (cluster.shape, distance_to_centroid.shape)

    return pruned_ids, Nj, Pj
d_intra, d_inter = compute_dintra_dinter(kmeans)
seed=1
for N in [750, 1500, 2250, 3000, 4000, 5000, 6000, 7000, 8000, 9000]:
    pruned_ids, Nj, Pj = pruning(kmeans, d_intra, d_inter, N=N)
    np.save(f"./cache/clipembds_kmeans_pruned_ids_N_{int(N)}_seed_{seed}.npy", pruned_ids)
    print ("N", N, pruned_ids[:100], len(pruned_ids))
    # plt.scatter(d_intra,d_inter)
    # plt.show()
    # plt.plot(Pj) 



749 [ 1  3 11  9 10  9 10  3 10  9  3  9 11 10  7  9  9  1  9  1  5  1  9  2
 10  6  9 10  7 10 11  9 10  9 10  8 10 10  9 10 10  9  9  9 10  1  2 10
  8  9  9  9  9 10  2 10  8  9  1  9 10  9  4  4 10 10  1  1 10  9  9  5
  9 10  1  2  9  9  4  7  9  9  9  9 11  9 10  1 11 11  1 11 10 10  9  1
 10 11  1  1]
N 750 [44242, 69027, 30032, 59255, 38636, 44284, 13895, 53332, 48513, 717, 68033, 71822, 52208, 7699, 63815, 12648, 7831, 3060, 66113, 72531, 53740, 55592, 14275, 1466, 24390, 59465, 30245, 24067, 9578, 45086, 6412, 5969, 15093, 67649, 55525, 12873, 42594, 12700, 3177, 12948, 21210, 16302, 65348, 21570, 10915, 55048, 41555, 26509, 40089, 30519, 22836, 27785, 57729, 56606, 2701, 47017, 71330, 15573, 61949, 61113, 8758, 11389, 60730, 49413, 48289, 47658, 24921, 37034, 50071, 22294, 7478, 32227, 25441, 50466, 30474, 2422, 52603, 36613, 13980, 39985, 29936, 32926, 61241, 59849, 6362, 26742, 53974, 17877, 20287, 18858, 16532, 14629, 6044, 6991, 55277, 27131, 14637, 65758, 38234, 48380] 

In [17]:
for N in [750, 1500, 2250, 3000, 4000, 5000, 6000, 7000, 8000, 9000]:
    random_ids = np.random.choice(keys, N, replace=False)
    np.save(f"./cache/random_clipembds_kmeans_pruned_ids_N_{int(N)}_seed_{seed}.npy", random_ids)
    print ("N", random_ids[:100], len(random_ids), len(np.unique(random_ids)))
    # plt.scatter(d_intra,d_inter)
    # plt.show()
    # plt.plot(Pj) 



N [33976 23211 34413 18315 56366 55142 35380 66046 18120 20935  5877  5044
 57549 15662 72201 36215 47470 43722 22012 61169 41924 60496   140 70426
 41015 30582 49723 53500 58155 52259 53124 37406 24647 68553 54111 70462
 54895 42338 23443 14873 15396 21202 14674  5582  3611 21788  1410 41425
 10054 50963 68884 65372 40682 38634 58111  9373  3625 38898 15567 13309
 13716 59850 67405 47639 57701 48281 38921 25098 33508 50553  9456 70077
 29812 70162  6329 31675 67919 41428  6915  7502 53905 46647 60790 13741
 52701 51008 32501  8790 17513  1395 28300 58539 25323 63194 68234 71777
 32076 40704 12085 66579] 750 750
N [38107 65255 28928 69085  7530 68008 52777 46993 48539 19435 65628 45372
 40608 17714 23479 70434 30275 17791  8486   204  6877 63072  8596 38995
 45994 70631  1493 25642 61948 45255 37267 26146 66329 64417  9224 59174
 71238 44404 38677  4033 22788  6750 38095 63528 64001 56244 48819 54568
 41935  6768 28294 45352 50963 70826 16998 47964 69890 16302  6873  2202
 45149  4360 

# Coreset CLIP

In [ ]:
idxs = []
voxel_ids = []
_voxels = torch.Tensor(_voxels).to('cuda:0')
pdist = torch.nn.PairwiseDistance(p=2,keepdim=True)
# fill with infinity

for i in range(len(_voxels)):
    if len(idxs) == 0:
        idxs.append(np.random.choice(len(_voxels), 1).item())
        voxel_ids.append(keys[idxs[0]])
        already_selected_voxels = _voxels[idxs].unsqueeze(0) # get selected voxels 
        dist = torch.cdist(_voxels.unsqueeze(0), already_selected_voxels).squeeze(0) 
        min_dist = torch.min(dist, dim=1).values
        print (dist.shape, min_dist.shape, min_dist)
        continue 
    argmax = torch.argmax(min_dist)
    assert argmax not in idxs, "Error, already selected"
    print (argmax, min_dist.shape, min_dist[argmax])
    idxs.append(argmax.item()) # add to selected 
    voxel_ids.append(keys[argmax]) # add to selected
    print (len(idxs))
    selected_voxels = _voxels[argmax].unsqueeze(0).unsqueeze(1) # get selected voxels 
    dist = torch.cdist(_voxels.unsqueeze(0), selected_voxels).flatten()
    # print ("d",dist.shape,min_dist.shape)
    min_dist = torch.minimum(dist, min_dist)
    # print ("selected_voxels.shape",selected_voxels.shape,  "mdist", mdist.shape, "min_dist", min_dist.shape)
    
    # if i > 10: break
    # already_selected_voxels = _voxels[idxs].unsqueeze(0) # get selected voxels
    # # print (len(idxs))
    # not_ids = list(set(range(len(_voxels))) - set(tuple(idxs))) # get not selected voxels ids
    # not_selected_voxels = _voxels[not_ids].unsqueeze(0) # get not selected voxels
    # print (not_selected_voxels.shape, already_selected_voxels.shape)
    # dist = torch.cdist(not_selected_voxels, already_selected_voxels).squeeze(0)
    
    # min_dist = torch.min(dist, dim=1)
    # argmax = torch.argmax(min_dist.values)
    # # print (dist,dist.shape, min_dist.values, argmax, min_dist.values[argmax])
    # assert not_ids[argmax] not in idxs, "Error, already selected"
    # idxs.append(not_ids[argmax]) # add to selected
    # voxel_ids.append(keys[not_ids[argmax]]) # add to selected
    



In [13]:
idxs = []
clip_ids = []
clip_embds = torch.Tensor(clip_embds).to('cuda:0').reshape(clip_embds.shape[0],-1)
print (clip_embds.shape)
pdist = torch.nn.PairwiseDistance(p=2,keepdim=True)
# fill with infinity

for i in range(len(clip_embds)):
    if len(idxs) == 0:
        idxs.append(np.random.choice(len(clip_embds), 1).item())
        clip_ids.append(keys[idxs[0]])
        already_selected_clip_embds = clip_embds[idxs]  # get selected voxels 
        dist = torch.cdist(clip_embds.unsqueeze(0), already_selected_clip_embds).squeeze(0)  
        # dist = []
        # for batch_ids in range(0, 9000, 300):
        #     img_batch = clip_embds[batch_ids:batch_ids+300] 
        #     dist.append(torch.cdist(img_batch.unsqueeze(0), already_selected_clip_embds).squeeze(0))
        # dist = torch.cat(dist, dim=0)

        min_dist = torch.min(dist, dim=1).values
        print (dist.shape, min_dist.shape, min_dist)
        continue 
    argmax = torch.argmax(min_dist)
    assert argmax not in idxs, "Error, already selected"
    print (argmax, min_dist.shape, min_dist[argmax])
    idxs.append(argmax.item()) # add to selected 
    clip_ids.append(keys[argmax]) # add to selected
    print (len(idxs))
    selected_clip_embds = clip_embds[argmax].unsqueeze(0).unsqueeze(1) # get selected voxels 
    dist = torch.cdist(clip_embds.unsqueeze(0), selected_clip_embds).flatten()
    # print ("d",dist.shape,min_dist.shape)
    min_dist = torch.minimum(dist, min_dist) 

torch.Size([9000, 197376])
torch.Size([9000, 1]) torch.Size([9000]) tensor([232.2396, 230.3910, 225.2516,  ..., 227.8880, 226.0505, 230.4500],
       device='cuda:0')
tensor(1845, device='cuda:0') torch.Size([9000]) tensor(241.7144, device='cuda:0')
2
tensor(7653, device='cuda:0') torch.Size([9000]) tensor(241.5125, device='cuda:0')
3
tensor(8184, device='cuda:0') torch.Size([9000]) tensor(241.3699, device='cuda:0')
4
tensor(7567, device='cuda:0') torch.Size([9000]) tensor(240.5834, device='cuda:0')
5
tensor(5667, device='cuda:0') torch.Size([9000]) tensor(240.4991, device='cuda:0')
6
tensor(2773, device='cuda:0') torch.Size([9000]) tensor(240.3896, device='cuda:0')
7
tensor(407, device='cuda:0') torch.Size([9000]) tensor(240.3240, device='cuda:0')
8
tensor(2067, device='cuda:0') torch.Size([9000]) tensor(240.1305, device='cuda:0')
9
tensor(953, device='cuda:0') torch.Size([9000]) tensor(240.0770, device='cuda:0')
10
tensor(536, device='cuda:0') torch.Size([9000]) tensor(240.0055, devi